Tasklist for this week's excercise:

- [x] Load data to DB
- [x] Read data from DB in iteration
- [x] Check how much duplicate rows are present and remove the duplicates
- [x] Get tags count and plot to find out mode frequent tags
- [x] Get # of tags per question count 
- [ ] Preprocessing Body: Remove html tags, spl cahrs, , lowercase all, stemming and lemmitization
- [ ] Define vector
- [ ] Define labels
- [ ] Splitting the data
- [ ] Define ML models
- [ ] Train, Test

Download the data from [here](https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction)

### Load the data from excel to sqlite database

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from wordcloud import WordCloud
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

### 1. Load the data from excel to sqlite database

In [2]:
%%time

chunksize = 180000
if not os.path.isfile("data/train.db"):
    disk_engine = create_engine("sqlite:///data/train.db")
    index_start = 1
    j=0
    for df in pd.read_csv('data/Train.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
        df.index+= index_start
        j+=1
        print(f"{j*chunksize} rows")
        df.to_sql('data', disk_engine, if_exists='append')
        index_start = df.index[-1] + 1

CPU times: user 10 µs, sys: 5 µs, total: 15 µs
Wall time: 17.6 µs


In [3]:
def execute(query, apply_chunk=True, delete=False):
    with sqlite3.connect('data/train.db') as con:
        if apply_chunk and not delete:
            data = pd.read_sql(query, con, chunksize= chunksize)
        elif delete == True:
            data = con.execute(query)
        else:
            data = pd.read_sql(query, con)
    return data    

#### 1.1 Get row count from db

In [ ]:
def get_row_count():
    num_rows = execute('SELECT COUNT(*) FROM data')
    count_rows = 0
    for i in num_rows:
        count_rows +=i.values
    return count_rows

In [ ]:
%%time

count_rows = int(get_row_count())
count_rows

### 2. Preprocessing the data

#### 2.1 Capturing the duplicate row counts

In [ ]:
def print_dup_rows_cnt():
    df = execute('SELECT Title, Body, Tags, COUNT(*) as cnt_dup FROM data GROUP BY Title, Body, Tags HAVING cnt_dup > 1')
    count_dict = {}
    for i in df:
        new_dict = dict(i.cnt_dup.value_counts())
        for k,v in new_dict.items():
            if k not in count_dict:
                count_dict[k] = v
            else:
                count_dict[k] += v
    dup_total = sum({k:v for k,v in count_dict.items() if k != 1}.values())
    print(f"{dup_total/int(count_rows) * 100} % of dataset has duplicates.")

In [ ]:
%%time

print_dup_rows_cnt()

#### 2.2 Removing duplicate rows

In [ ]:
%%time

execute('DELETE FROM data WHERE ID NOT IN (SELECT MAX(ID) FROM data GROUP BY Title, Body, Tags)', delete=True)

In [ ]:
%%time

print(int(get_row_count()))

In [ ]:
%%time

print_dup_rows_cnt()

### 3. Exploring the data 

#### 3.1 Getting the tags frequency

In [ ]:
%%time


df = execute('SELECT Tags FROM data')
tags_dict = {}
for i in df:
    tag_list_list = i.Tags.apply(lambda x: x.split(' ') if x else []).values
    for tag_list in tag_list_list:
        for tag_name in tag_list:
            if tag_name not in tags_dict:
                tags_dict[tag_name] = 1
            else:
                tags_dict[tag_name]+=1

In [ ]:
tags_dict

In [ ]:
tags_to_idx = {k:i for i,(k,v) in enumerate(tags_dict.items())}
tags_to_idx

In [ ]:
df_tag_cnt = pd.DataFrame(columns=['tags', 'counts'])
df_tag_cnt.tags = tags_dict.keys()
df_tag_cnt.counts = tags_dict.values()

In [ ]:
df_tag_cnt

In [ ]:
tag_df_sorted = df_tag_cnt.sort_values(['counts'], ascending=False)
tag_counts = tag_df_sorted['counts'].values

In [ ]:
plt.plot(tag_counts)
plt.title("Distribution of number of times tag appeared questions")
plt.grid()
plt.xlabel("Tag number")
plt.ylabel("Number of times tag appeared")
plt.show()

In [ ]:
plt.plot(tag_counts[0:500])
plt.title("Distribution of number of times tag appeared questions")
plt.grid()
plt.xlabel("Tag number")
plt.ylabel("Number of times tag appeared")
plt.show()

In [ ]:
plt.plot(tag_counts[0:100], c='b')
plt.scatter(x=list(range(0,100,5)), y=tag_counts[0:100:5], c='orange', label="quantiles with 0.05 intervals")
# quantiles with 0.25 difference
plt.scatter(x=list(range(0,100,25)), y=tag_counts[0:100:25], c='m', label = "quantiles with 0.25 intervals")

for x,y in zip(list(range(0,100,25)), tag_counts[0:100:25]):
    plt.annotate(text="({} , {})".format(x,y), xy=(x,y), xytext=(x-0.05, y+500))

plt.title('first 100 tags: Distribution of number of times tag appeared questions')
plt.grid()
plt.xlabel("Tag number")
plt.ylabel("Number of times tag appeared")
plt.legend()
plt.show()
print(len(tag_counts[0:100:5]), tag_counts[0:100:5])

In [ ]:
df_tag_cnt.sort_values(['counts'], ascending=False)

#### 3.2 Getting the tags per question count

In [ ]:
%%time


df = execute('SELECT Tags FROM data')
tags_cnt_per_q = []
for i in df:
    tag_list_list = i.Tags.apply(lambda x: x.split(' ') if x else []).values
    for tag_list in tag_list_list:
        tags_cnt_per_q.append(len(tag_list))

In [ ]:
len(tags_cnt_per_q)

In [ ]:
sns.countplot(x=tags_cnt_per_q ,palette='gist_rainbow')
plt.title("Number of tags in the questions ")
plt.xlabel("Number of Tags")
plt.ylabel("Number of questions")
plt.show()

In [ ]:
tags_cnt_per_q

In [ ]:
tags_dict

In [ ]:
tup = dict(tags_dict.items())
wordcloud = WordCloud(    background_color='black',
                          width=1600,
                          height=800,
                    ).generate_from_frequencies(tup)

fig = plt.figure(figsize=(30,20))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
i=np.arange(30)
tag_df_sorted.head(30).plot(kind='bar')
plt.title('Frequency of top 20 tags')
plt.xticks(i, tag_df_sorted['tags'][:30])
plt.xlabel('Tags')
plt.ylabel('Counts')
plt.show()

In [4]:
def striphtml(data):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(data))
    return cleantext
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

In [24]:
df = execute('SELECT * FROM data')
disk_engine = create_engine("sqlite:///data/train.db")
for i in df:
    print(len(i))
    i.Body = i.Body.apply(striphtml)
    i.to_sql('data_processed', disk_engine, if_exists='append')

180000


OperationalError: (sqlite3.OperationalError) database is locked
[SQL: 
CREATE TABLE data_processed (
	level_0 BIGINT, 
	"index" BIGINT, 
	"Id" BIGINT, 
	"Title" TEXT, 
	"Body" TEXT, 
	"Tags" TEXT
)

]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [19]:
i.to_sql('data_processed', disk_engine, if_exists='append')

In [25]:
df.close()

In [29]:
df = execute('SELECT * FROM data')

In [30]:
for i in df:
    print(len(i))

180000
180000
180000
180000


KeyboardInterrupt: 

In [31]:
con = sqlite3.connect('data/train.db')

In [ ]:
con.execute()